# 手工初级练丹 - level UP!

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

## 数据读取

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='./mnist.pnz')

## 数据预处理

和初级练丹一致，slice切片->scaling数值区间，类型转换->打散->做batch <br/>

第一步定义一个是数据预处理函数对输入和真值的类型和数值范围进行转换

In [ ]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255. # cast is ok; but convert_to_tensor is not working-> uint8 can't be converted to float32 tensor
    y = tf.cast(y, dtype=tf.int64)
    return x, y

接下来就是做数据切片了。第一个是训练集合。

In [ ]:
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(preprocess).shuffle(10000).batch(100)

测试集,不需要打散（搅拌？乱序）shuttle了

In [ ]:
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocess).batch(100)

## 搭练丹炉了---网络

这里直接使用了keras封装的Dense层类和Sequential序列全连接层类

In [ ]:
network = keras.Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(10)
])
network.build(input_shape=[None, 28*28]) # 给定输入层的结构从而初始化网络
network.summary() # 打印出网络结构

optimizer = optimizers.Adam(lr=1e-3) # 参数更新优化器，LR learnning rate

主函数

In [ ]:
def main():
    for epoch in range(50):
        # Training...
        for step, (x, y) in enumerate(train_db):
            x = tf.reshape(x, [-1, 28*28])
            with tf.GradientTape() as tape:
                logits = network(x)
                y_onehot = tf.one_hot(y, depth=10)
                loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)

            grads = tape.gradient(loss, network.trainable_variables)
            optimizer.apply_gradients(zip(grads, network.trainable_variables))

            if step % 100 == 0 : 
                print(epoch, step, 'loss:', float(tf.reduce_mean(loss)))

        # Testing...
        total_correct, total_num = 0, 0
        for step, (x, y) in enumerate(test_db):
            x = tf.reshape(x, [-1, 28*28])
            out = network(x)

            prob = tf.nn.softmax(out, axis=1)
            pred = tf.argmax(prob, axis=1)
            
            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)
            
            total_correct += int(correct)
            total_num += x.shape[0]

        print('test acc:', total_correct * 100 / total_num, '%')

判断脚本进入main的位置，防止全局变量污染

In [ ]:
if __name__ == '__main__':
    main()